# LAB 04: 


## Task 1: Implement Node Class

Write class Node that has the following attributes:

- state
- parent
- action
- path_cost


In [1]:
class Node:
    def __init__(self, state, parent, actions, totalcost):
        self.state = state
        self.parent = parent
        self.actions = actions
        self.totalcost = totalcost

## Task 2: Implemnet Graphs using Node Class

Create different graphs using the Node class. Use dictionaries to store the graph nodes. The keys of the dictionaries are the names of the nodes and the values are the Node objects.


In [8]:
romania = {
    "Arad": Node("Arad", None, [("Zerind", 75), ("Sibiu", 140), ("Timisoara", 118)], 0),
    "Zerind": Node("Zerind", None, [("Arad", 75), ("Oradea", 71)], 0),
    "Oradea": Node("Oradea", None, [("Zerind", 71), ("Sibiu", 151)], 0),
    "Sibiu": Node("Sibiu", None, [("Arad", 140), ("Oradea", 151), ("Fagaras", 99), ("Rimnicu Vilcea", 80)], 0),
    "Timisoara": Node("Timisoara", None, [("Arad", 118), ("Lugoj", 111)], 0),
    "Lugoj": Node("Lugoj", None, [("Timisoara", 111), ("Mehadia", 70)], 0),
    "Mehadia": Node("Mehadia", None, [("Lugoj", 70), ("Drobeta", 75)], 0),
    "Drobeta": Node("Drobeta", None, [("Mehadia", 75), ("Craiova", 120)], 0),
    "Craiova": Node("Craiova", None, [("Drobeta", 120), ("Rimnicu Vilcea", 146), ("Pitesti", 138)], 0),
    "Rimnicu Vilcea": Node("Rimnicu Vilcea", None, [("Sibiu", 80), ("Craiova", 146), ("Pitesti", 97)], 0),
    "Fagaras": Node("Fagaras", None, [("Sibiu", 99), ("Bucharest", 211)], 0),
    "Pitesti": Node("Pitesti", None, [("Rimnicu Vilcea", 97), ("Craiova", 138), ("Bucharest", 101)], 0),
    "Bucharest": Node("Bucharest", None, [("Fagaras", 211), ("Pitesti", 101), ("Giurgiu", 90), ("Urziceni", 85)], 0),
    "Giurgiu": Node("Giurgiu", None, [("Bucharest", 90)], 0),
    "Urziceni": Node("Urziceni", None, [("Bucharest", 85), ("Hirsova", 98), ("Vaslui", 142)], 0),
    "Hirsova": Node("Hirsova", None, [("Urziceni", 98), ("Eforie", 86)], 0),
    "Eforie": Node("Eforie", None, [("Hirsova", 86)], 0),
    "Vaslui": Node("Vaslui", None, [("Urziceni", 142), ("Iasi", 92)], 0),
    "Iasi": Node("Iasi", None, [("Vaslui", 92), ("Neamt", 87)], 0),
    "Neamt": Node("Neamt", None, [("Iasi", 87)], 0),
}


## Task 3: Implement Graph as Tree

Write a function that takes a graph and a start node and returns the tree resulted from the graph search algorithm. The function should return the root node of the tree.

In [21]:
from collections import deque

def BFS_tree(graph, start):
    queue = deque([(start, None, 0)])  # (node, parent, cost)
    visited = set()
    bfs_tree = {start: [(None, 0)]}  # Initialize with Arad and cost 0

    while queue:
        current, parent, cost = queue.popleft()

        if current not in visited:
            visited.add(current)
            if parent is not None:
                if current not in bfs_tree:
                    bfs_tree[current] = []
                bfs_tree[current].append((parent, cost))
            
            for neighbor, edge_cost in graph[current].actions:
                if neighbor not in visited:
                    queue.append((neighbor, current, edge_cost))

    return bfs_tree

# Usage example:
start_node = "Arad"  # Replace with the desired start node

bfs_tree = BFS_tree(romania, start_node)

# Print the BFS tree
for node, neighbors in bfs_tree.items():
    print(f"{node}: {neighbors}")


Arad: [(None, 0)]
Zerind: [('Arad', 75)]
Sibiu: [('Arad', 140)]
Timisoara: [('Arad', 118)]
Oradea: [('Zerind', 71)]
Fagaras: [('Sibiu', 99)]
Rimnicu Vilcea: [('Sibiu', 80)]
Lugoj: [('Timisoara', 111)]
Bucharest: [('Fagaras', 211)]
Craiova: [('Rimnicu Vilcea', 146)]
Pitesti: [('Rimnicu Vilcea', 97)]
Mehadia: [('Lugoj', 70)]
Giurgiu: [('Bucharest', 90)]
Urziceni: [('Bucharest', 85)]
Drobeta: [('Craiova', 120)]
Hirsova: [('Urziceni', 98)]
Vaslui: [('Urziceni', 142)]
Eforie: [('Hirsova', 86)]
Iasi: [('Vaslui', 92)]
Neamt: [('Iasi', 87)]


In [25]:
from anytree import Node, RenderTree

def BFS_tree_search(graph, start, goal):
    queue = [(start, None)]  # (node, parent)
    visited = set()
    search_tree = {start: None}  # Initialize with start node and no parent

    while queue:
        current, parent = queue.pop(0)

        if current == goal:
            break  # Goal node found, exit the loop

        if current not in visited:
            visited.add(current)
            
            for neighbor, _ in graph[current].actions:
                if neighbor not in visited:
                    queue.append((neighbor, current))
                    if neighbor not in search_tree:
                        search_tree[neighbor] = current  # Set the parent of the neighbor

    return search_tree

# Usage example:
start_node = "Arad"  # Replace with the desired start node
goal_node = "Bucharest"  # Replace with the desired goal node

# Run the BFS search and create the search tree
search_tree = BFS_tree_search(romania, start_node, goal_node)

# Create the tree representation from the search tree
def create_tree_representation(tree, current_node):
    if current_node in tree:
        node = Node(current_node)
        for neighbor, parent in tree.items():
            if parent == current_node:
                child = create_tree_representation(tree, neighbor)
                child.parent = node
        return node

# Create the tree representation
tree_representation = create_tree_representation(search_tree, start_node)

# Print the tree representation
for pre, fill, node in RenderTree(tree_representation):
    print(f"{pre}{node.name}")


Arad
├── Zerind
│   └── Oradea
├── Sibiu
│   ├── Fagaras
│   │   └── Bucharest
│   └── Rimnicu Vilcea
│       ├── Craiova
│       └── Pitesti
└── Timisoara
    └── Lugoj
        └── Mehadia
